## Simulation: Cortical Gamma Oscillations in Schizophrenia
This simulation models a local cortical circuit (a small patch of brain tissue) to investigate the biological causes of Schizophrenia.
Specifically, it is designed to test the "NMDA Receptor Hypofunction Hypothesis," which suggests that schizophrenia symptoms arise because specific glutamate receptors (NMDA) are underactive, particularly on inhibitory neurons.

The network consists of two neuronal populations that generate a PING (Pyramidal–Interneuron Network Gamma) rhythm.
- Pyramidal Cells (E-cells)Excitatory neurons ($N_E = 200$). These cells provide the recurrent excitation and drive the network dynamics.
- Fast-Spiking Interneurons (I-cells)Inhibitory parvalbumin-positive ($PV^+$) neurons ($N_I = 50$). These neurons act as the pacemaker of the circuit, firing rapidly to synchronize pyramidal-cell activity.
- Connectivity is sparse and random, defined by the connection probabilities$$P_{EE},\; P_{EI},\; P_{IE},\; P_{II}, $$corresponding to excitatory–excitatory, excitatory–inhibitory, inhibitory–excitatory, and inhibitory–inhibitory synapses.

All neurons are described by quadratic integrate-and-fire (QIF) dynamics, which are mathematically equivalent to the Izhikevich model near spike initiation and provide a more realistic description of excitability than leaky integrate-and-fire models.The membrane potential evolves according to$$C \frac{dV}{dt} = g_L \frac{(V - E_L)(V - V_T)}{V_T - E_L} - I_{\text{syn}} - I_{\text{adapt}} + I_{\text{ext}} + I_{\text{noise}} .$$Intrinsic ParametersEach neuron is characterized by:$$C,\; g_L,\; E_L,\; V_T,\; V_R .$$A key biological asymmetry is the difference in leak conductance:$$g_L^{(I)} = 0.5 \;\text{mS/cm}^2, \qquad g_L^{(E)} = 0.05 \;\text{mS/cm}^2,$$making interneurons significantly less excitable at rest and strongly dependent on synaptic input

Pyramidal neurons include a slow spike-frequency adaptation variable $z$:$$\frac{dz}{dt} = -a z,$$with an increment at each spike,$$z \leftarrow z + d .$$This mechanism limits excessive firing and stabilizes gamma oscillations. Interneurons do not exhibit adaptation

Synaptic DynamicsSynaptic transmission is mediated by AMPA, NMDA, and GABA$_A$ receptors.AMPA and GABA SynapsesFast excitatory (AMPA) and inhibitory (GABA) currents are modeled as linear conductances:$$I = g \cdot s \cdot (V - E_{\text{rev}}),$$where $g$ is the maximal conductance, $s$ the synaptic gating variable, and $E_{\text{rev}}$ the reversal potential.NMDA Synapses (Voltage-Dependent Magnesium Block)NMDA currents include a voltage-dependent magnesium block following Jahr & Stevens (1990):$$I_{\text{NMDA}} = g_{\text{NMDA}} \cdot s \cdot (V - E_{\text{exc}}) \left( \frac{1}{1 + [Mg^{2+}] \cdot \exp(-0.062 \cdot V / 3.57)} \right).$$At hyperpolarized potentials, $Mg^{2+}$ blocks the NMDA channel; depolarization relieves this block, introducing a strong nonlinearity in synaptic excitation.5. Synaptic ConductancesThe model includes the following synaptic conductances:$$g_{EE},\; g_{NE},\; g_{EI},\; g_{NI},\; g_{IE},\; g_{II},$$corresponding to AMPA, NMDA, and GABA connections between excitatory and inhibitory populations. Synaptic time constants and reversal potentials are receptor-specific and fixed.


## model initialization

In [27]:
import numpy as np
from brian2 import *

In [28]:

# Unit conversions (paper uses per cm^2; Brian2 wants SI per m^2)

# 1 cm^2 = 1e-4 m^2
# so X / cm^2  ==  X * 1e4 / m^2
#
# 1 uA/cm^2 = 1e-6 A * 1e4 / m^2 = 1e-2 A/m^2
# 1 mS/cm^2 = 1e-3 S * 1e4 / m^2 = 10 S/m^2
# 1 uF/cm^2 = 1e-6 F * 1e4 / m^2 = 1e-2 F/m^2
UA_PER_CM2_TO_SI = 1e-2 * amp / meter**2
MS_PER_CM2_TO_SI = 10.0 * siemens / meter**2
UF_PER_CM2_TO_SI = 1e-2 * farad / meter**2


#### Network size + connectivity

In [29]:

N_E, N_I = 200, 50
P_EE, P_EI, P_IE, P_II = 0.10, 0.40, 0.50, 0.60

#### parameters and equations 

In [30]:
# Capacitance
C_E = 1.0 * UF_PER_CM2_TO_SI
C_I = 1.0 * UF_PER_CM2_TO_SI

# Leak
EL  = -65.0 * mV
gL_E = 0.05 * MS_PER_CM2_TO_SI
gL_I = 0.50 * MS_PER_CM2_TO_SI

# Thresholds / reset
VT_E = -45.0 * mV
VT_I = -30.0 * mV
VR   = -52.0 * mV
V_peak = 20 * mV


# Adaptation (E only)
a = 0.01 / ms          # paper form uses fixed decay
d = 0.2 * UA_PER_CM2_TO_SI

# Synaptic time constants
tau_nmda = 80.0 * ms
tau_ampa_EE = 3.0 * ms
tau_ampa_EI = 1.0 * ms
tau_gaba = 2.0 * ms

# Reversal potentials
E_AMPA = 0.0 * mV
E_NMDA = 0.0 * mV
E_GABA = -70.0 * mV

# Synaptic conductances (Table 2)

gNE = 0.008 * MS_PER_CM2_TO_SI
gNI = 0.008 * MS_PER_CM2_TO_SI
gEE = 0.10  * MS_PER_CM2_TO_SI
gEI = 0.08  * MS_PER_CM2_TO_SI
gIE = 0.25  * MS_PER_CM2_TO_SI
gII = 0.10  * MS_PER_CM2_TO_SI

# Magnesium concentration
Mg = 1.0

# External tonic drive
I_app_E = 4.0 * UA_PER_CM2_TO_SI
I_app_I = 0.0 * UA_PER_CM2_TO_SI

eqs_E = """
# Intrinsic quadratic term (as in the paper's QIF-like formulation)
I_intrinsic = gL * (v - EL) * (v - VT) / (VT - EL) : amp/meter**2
I_adapt = z : amp/meter**2

# NMDA magnesium block
B = 1.0 / (1.0 + (Mg * exp(-0.062 * (v/mV)) / 3.57)) : 1

# Synaptic currents: E receives from E (AMPA+NMDA) and from I (GABA)
I_ampa = gEE * sA_EE * (v - E_AMPA) : amp/meter**2
I_nmda = gNE * sN_EE * B * (v - E_NMDA) : amp/meter**2
I_gaba = gIE * sG_IE * (v - E_GABA) : amp/meter**2
I_syn = I_ampa + I_nmda + I_gaba : amp/meter**2

# Membrane
dv/dt = (I_app + I_intrinsic - I_adapt - I_syn) / C : volt

# Adaptation
dz/dt = -a * z : amp/meter**2

# Synaptic gate decays
dsA_EE/dt = -sA_EE / tau_ampa_EE : 1
dsN_EE/dt = -sN_EE / tau_nmda : 1
dsG_IE/dt = -sG_IE / tau_gaba : 1

# Per-neuron parameters / inputs
I_app : amp/meter**2
C : farad/meter**2
gL : siemens/meter**2
VT : volt
"""

eqs_I = """
I_intrinsic = gL * (v - EL) * (v - VT) / (VT - EL) : amp/meter**2
I_adapt = 0*amp/meter**2 : amp/meter**2  # no adaptation current in I

B = 1.0 / (1.0 + (Mg * exp(-0.062 * (v/mV)) / 3.57)) : 1

# Synaptic currents: I receives from E (AMPA+NMDA) and from I (GABA)
I_ampa = gEI * sA_EI * (v - E_AMPA) : amp/meter**2
I_nmda = gNI * sN_EI * B * (v - E_NMDA) : amp/meter**2
I_gaba = gII * sG_II * (v - E_GABA) : amp/meter**2
I_syn = I_ampa + I_nmda + I_gaba : amp/meter**2

dv/dt = (I_app + I_intrinsic - I_adapt - I_syn) / C : volt

# Gate decays
dsA_EI/dt = -sA_EI / tau_ampa_EI : 1
dsN_EI/dt = -sN_EI / tau_nmda : 1
dsG_II/dt = -sG_II / tau_gaba : 1

I_app : amp/meter**2
C : farad/meter**2
gL : siemens/meter**2
VT : volt
"""



#### population and synapse

In [43]:
def network(alpha_EI, dt, rng_seed=0,gNI_mS_cm2=0.008):
    global gNI  # because eqs_I refers to the Python symbol gNI
    start_scope()
    np.random.seed(rng_seed)
    seed(rng_seed) 
    
    gNI = gNI_mS_cm2 * MS_PER_CM2_TO_SI

    defaultclock.dt = dt * ms
    method = "euler"
    

    E = NeuronGroup(
        N_E, eqs_E,
        threshold="v >= V_peak",
        reset="v = VR; z += d",
        method=method,
        name="E"
    )

    I = NeuronGroup(
        N_I, eqs_I,
        threshold="v >= V_peak",
        reset="v = VR",          # no adaptation jump in I
        method=method,
        name="I"
    )

    # Set per-population intrinsic parameters
    E.C = C_E
    E.gL = gL_E
    E.VT = VT_E
    E.I_app = I_app_E

    I.C = C_I
    I.gL = gL_I
    I.VT = VT_I
    I.I_app = I_app_I

    # Initial conditions
    E.v = EL + 1*mV*randn(N_E)
    E.z = 0.0 * amp/meter**2
    E.sA_EE = 0.0
    E.sN_EE = 0.0
    E.sG_IE = 0.0

    I.v = EL + 1*mV*randn(N_I)
    I.sA_EI = 0.0
    I.sN_EI = 0.0
    I.sG_II = 0.0

    # Weights (dimensionless gate jumps)
    K_EE = P_EE * (N_E - 1)
    K_EI = P_EI * N_E
    K_IE = P_IE * N_I
    K_II = P_II * (N_I - 1)

    wA_EE = 1.0 / K_EE
    wN_EE = 1.0 / K_EE
    wA_EI = 1.0 / K_EI
    wN_EI = 1.0 / K_EI
    wG_IE = 1.0 / K_IE
    wG_II = 1.0 / K_II

    # E -> E (paired AMPA+NMDA on same edges)
    S_EE = Synapses(E, E, model="wA:1\nwN:1", on_pre="sA_EE_post += wA; sN_EE_post += wN")
    S_EE.connect(condition='i != j', p=P_EE)
    S_EE.wA = wA_EE
    S_EE.wN = wN_EE

    # E -> I
    S_EI = Synapses(E, I, model="wA:1\nwN:1", on_pre="sA_EI_post += wA; sN_EI_post += wN")
    S_EI.connect(p=P_EI)


    S_EI.wA = alpha_EI
    S_EI.wN = alpha_EI

    # I -> E
    S_IE = Synapses(I, E, model="wG:1", on_pre="sG_IE_post += wG")
    S_IE.connect(p=P_IE)
    S_IE.wG = wG_IE

    # I -> I
    S_II = Synapses(I, I, model="wG:1", on_pre="sG_II_post += wG")
    S_II.connect(condition='i != j', p=P_II)
    S_II.wG = wG_II

    for S in (S_EE, S_EI, S_IE, S_II):
        S.delay = 0.5 * ms


    # ----------------------------
    # Recording
    # ----------------------------
    spE = SpikeMonitor(E)
    spI = SpikeMonitor(I)

    vE = StateMonitor(E, "v", record=range(5))
    vI = StateMonitor(I, "v", record=range(5))

    T = 2000*ms
    run(T)

    t0 = 200*ms
    rate_E_ss = np.sum(spE.t >= t0) / ((T - t0)/second) / N_E
    rate_I_ss = np.sum(spI.t >= t0) / ((T - t0)/second) / N_I
    vmax0_mV = float(np.max(vE.v[0] / mV))

    return dict(alpha_EI=alpha_EI, dt=float(dt/ms),
                rate_E_ss=float(rate_E_ss), rate_I_ss=float(rate_I_ss),
                I_app_E=float(E.I_app[0] / (amp/meter**2)),
                I_app_I=float(I.I_app[0] / (amp/meter**2)),
                vmax0_mV=vmax0_mV)
    

## Implementation tuning 




Before reproducing the experiments in the paper, we performed a limited amount of **implementation tuning**. This tuning was **not part of the scientific results**, but was necessary to obtain a stable and numerically correct realization of the model.

In particular, we:
- verified **numerical convergence** with respect to the integration timestep (`dt`) and fixed it once firing rates were stable,
- introduced a single **synaptic scaling factor** (`alpha_EI`) to resolve an underspecified discretization detail (normalization of E→I synaptic gating increments), and selected a value that produced a stable excitatory–inhibitory balance,
- fixed all other model parameters exactly as specified in the paper.

These steps define a **single, fixed model instantiation** and do not correspond to parameter fitting or experimental manipulation.

After this implementation tuning was completed, all subsequent simulations follow the paper’s methodology and vary only **experimental control parameters**, such as external input to excitatory neurons and NMDA conductance onto inhibitory neurons (`gNI`), as described in the original study.

In [32]:
for alpha_EI in [0.30, 0.20, 0.15, 0.10]:
    out=network(alpha_EI=alpha_EI, dt=0.2, rng_seed=1)
    print(out)

{'alpha_EI': 0.3, 'dt': 200.0, 'rate_E_ss': 45.425, 'rate_I_ss': 141.05555555555554, 'vmax0_mV': 19.959983513459797}
{'alpha_EI': 0.2, 'dt': 200.0, 'rate_E_ss': 55.263888888888886, 'rate_I_ss': 107.17777777777778, 'vmax0_mV': 19.956900649597262}
{'alpha_EI': 0.15, 'dt': 200.0, 'rate_E_ss': 62.875, 'rate_I_ss': 80.52222222222221, 'vmax0_mV': 19.99164309824145}
{'alpha_EI': 0.1, 'dt': 200.0, 'rate_E_ss': 74.58333333333333, 'rate_I_ss': 39.34444444444444, 'vmax0_mV': 19.995486619291732}


We selected **`alpha_EI = 0.15`** because it lies in the interior of a balanced regime in which:
- both excitatory and inhibitory populations are active,
- inhibitory firing rates remain higher than excitatory firing rates,
- firing rates are not extreme,
- and the qualitative network behavior is robust to small parameter changes.




In [33]:
for dt in [0.02, 0.01, 0.005]:
    out = network(alpha_EI=0.15, dt=dt, rng_seed=1)
    print(out)

{'alpha_EI': 0.15, 'dt': 20.0, 'rate_E_ss': 63.02222222222222, 'rate_I_ss': 82.82222222222222, 'vmax0_mV': 19.99879770407278}
{'alpha_EI': 0.15, 'dt': 10.0, 'rate_E_ss': 63.019444444444446, 'rate_I_ss': 83.0, 'vmax0_mV': 19.99810280804032}
{'alpha_EI': 0.15, 'dt': 5.0, 'rate_E_ss': 63.03333333333333, 'rate_I_ss': 83.11111111111111, 'vmax0_mV': 19.99979188145056}


We selected **`dt = 0.02 ms`** because it lies in the interior of a numerically converged regime in which:
- steady-state firing rates of both excitatory and inhibitory populations are stable under timestep refinement,
- halving the timestep produces negligible changes in firing rates (≪ 1%),
- the qualitative dynamical regime of the network is preserved,
- and the timestep is sufficiently small to accurately resolve fast spike dynamics without unnecessary computational cost.


### Choice of the spike cutoff voltage (`V_peak`)

The spike cutoff voltage `V_peak` is a numerical parameter used to terminate the divergent trajectory of the quadratic integrate-and-fire model. It does not correspond to a biophysical action-potential peak and should therefore be chosen such that network observables are insensitive to its precise value.

We tested several values of `V_peak` (0 mV, 10 mV, and 20 mV) while keeping all other parameters fixed. Lower cutoff values led to systematically higher firing rates due to earlier spike detection in the discrete-time integration scheme.

We selected **`V_peak = 20 mV`** because it lies in a regime where firing rates are lowest and least sensitive to the precise cutoff value, providing the best approximation to the ideal infinite-voltage spike definition of the QIF model. All subsequent simulations fix `V_peak` at this value.


# 1. NMDA Conductance onto Fast-Spiking Interneurons

Parameter $g_{NI}$: NMDA synaptic conductance onto inhibitory (FSI) neurons.

How it is varied
- The parameter is swept over a wide range, including:
    - $g_{NI} = 0$ (complete NMDA removal)
    - Intermediate values (e.g., $0.006$–$0.03$)
    - High values (e.g., $0.054$)
    
Purpose: 
- To model NMDA hypofunction versus hyperfunction on FSIs.
- To test for non-monotonic (inverted-U) effects on gamma power

In [44]:
I_app_I = 0.5 * UA_PER_CM2_TO_SI 

for g in [0.0, 0.006, 0.012, 0.018, 0.024, 0.054]:
    
    out = network(alpha_EI=0.15, dt=0.02, rng_seed=4, gNI_mS_cm2=g)
    print(g, out["rate_E_ss"], out["rate_I_ss"], out["I_app_I"])




0.0 85.10277777777777 7.055555555555555 0.005
0.006 64.89722222222221 76.12222222222222 0.005
0.012 53.55277777777778 114.7 0.005
0.018 46.02777777777777 140.04444444444442 0.005
0.024 40.519444444444446 158.13333333333333 0.005
0.054 25.819444444444443 204.82222222222222 0.005


### Effect of NMDA conductance onto inhibitory neurons (`gNI`)

Increasing the NMDA conductance from excitatory to inhibitory neurons (`gNI`) produces a systematic shift in network balance. As `gNI` increases, inhibitory neurons receive stronger and more persistent excitation, leading to a marked increase in inhibitory firing rates. The resulting feedback inhibition progressively suppresses excitatory firing.

For very small `gNI`, inhibition is weak and excitatory neurons dominate the network dynamics. For large `gNI`, inhibition becomes overly strong and suppresses excitatory activity. The most relevant dynamical regime lies at intermediate `gNI`, where both populations are active and inhibitory firing rates exceed excitatory rates. This regime corresponds to the parameter range in which the paper reports robust gamma-band oscillations.
